In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

config = SparkConf().setMaster('local').setAppName('lab22')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

In [6]:
DATASET_PATH = '/content/gdrive/MyDrive/1800.csv'
lines = sc.textFile(DATASET_PATH)

def parseLine(line):
  fields = line.split(',')
  loc = fields[0]
  temp_type = fields[2]
  temperature = float(fields[3]) * 0.1 *(0.9/5.0) + 32.0
  return (loc, temp_type, temperature)

rdd = lines.map(parseLine)

In [ ]:
TMIN_rdd = rdd.filter(lambda x: x[1] == 'TMIN')
hide_TMin_Col = TMIN_rdd.map(lambda x: (x[0], x[2]))
result = hide_TMin_Col.reduceByKey(lambda x, y: min(x,y))
for temp in result.collect():
  print(temp)